## rotate.py

Moves all images in a given directory into four subdirectories, then labels rotates them accordingly.

In [2]:
import os
import random
import numpy as np
import PIL
from PIL import Image
from pathlib import Path
from tqdm import tqdm

In [3]:
augment = True
delete_originals = False

absolute_path_to_sources = "E:/GOMBOC_DATA/[PROCESSING]"

def make_folders(directory):
    Path(directory).mkdir(exist_ok=True)
    Path(directory + "/0").mkdir(exist_ok=True)
    Path(directory + "/90").mkdir(exist_ok=True)
    Path(directory + "/180").mkdir(exist_ok=True)
    Path(directory + "/270").mkdir(exist_ok=True)

Path("data").mkdir(exist_ok=True)
Path("data/test").mkdir(exist_ok=True)


for folder in Path(absolute_path_to_sources).glob("*"):
    np.random.seed(1337)
    random.seed(1337)

    dataset_folder_name = folder.name
    print("Processing folder:", dataset_folder_name)

    # Make folders
    for folder in ["test", "train", "validate"]:
        Path("data/" + folder).mkdir(exist_ok=True)
        write_path = "data/" + folder + "/" + dataset_folder_name
        make_folders(write_path)
    
    paths = Path(absolute_path_to_sources + "/" + dataset_folder_name).glob("*")
    files = np.array([x for x in paths if x.is_file()])
    np.random.shuffle(files)

    for i in tqdm(range(len(files))):
        file_path = files[i]
        try:
            img = Image.open(file_path)
        except:
            # print("Could not open image:", file_path)
            # print("Continuing...")
            continue

        rotation_degree = { 0: 0, 1: 90, 2: 180, 3: 270 } 
        rotations_set =  [0, 1, 2, 3] if augment else [random.randint(0, 3)]

        for rotations in rotations_set:
            degrees = rotation_degree[rotations]
            label = str(360 - degrees if degrees > 0 else 0)
            
            try:
                write_img = img.rotate(degrees, expand=True, resample=Image.Resampling.BICUBIC) if degrees > 0 else img.copy()
                file_name = Path(file_path).name

                # Write image to new folder
                if ((i + 1) / len(files) <= 0.1):
                    #Test
                    write_img.save("data/test/" + dataset_folder_name + "/" + label + "/" + file_name)
                elif ((i + 1) / len(files) <= 0.3):
                    # Validate
                    write_img.save("data/validate/" + dataset_folder_name + "/" + label + "/" + file_name)
                else:
                    # Train
                    write_img.save("data/train/" + dataset_folder_name + "/" + label + "/" + file_name)
                        
            except:
                continue
                #print("Could not write image:", write_path + "/" + label + "/" + file_name)
                #print("Continuing...")

        # Delete original image
        if (delete_originals):
            Path(file_path).unlink()

# cv2.imwrite("testout.JPEG", write_image)



Processing folder: illustrations


  1%|          | 58/5260 [00:19<24:50,  3.49it/s]  C:\Users\Curtis\AppData\Roaming\Python\Python310\site-packages\PIL\Image.py:3035: DecompressionBombWarning: Image size (121615285 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 19%|█▊        | 984/5260 [06:00<18:21,  3.88it/s]  C:\Users\Curtis\AppData\Roaming\Python\Python310\site-packages\PIL\Image.py:3035: DecompressionBombWarning: Image size (90315008 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 49%|████▉     | 2565/5260 [16:16<19:14,  2.33it/s]  C:\Users\Curtis\AppData\Roaming\Python\Python310\site-packages\PIL\Image.py:3035: DecompressionBombWarning: Image size (92878016 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 69%|██████▉   | 3633/5260 [23:07<14:37,  1.85it/s]  

## References
 - https://www.tensorflow.org/tutorials/load_data/images
 - https://stackoverflow.com/questions/17821945/how-to-preserve-image-quality-when-rotating-with-pil
